In [ ]:
# This code was modified from Hack Oregon's Neighborhood Development project
# https://github.com/hackoregon/neighborhoods-2018/tree/master/notebooks/campsites
# Many thanks to Hasan for collaborating on the data prep through Hack Oregon

In [3]:
import pyproj
from pyproj import transform

In [4]:
import pandas as pd
import numpy as np

In [10]:
# http://www.spatialreference.org/ref/epsg/3646/
or_north_plain = pyproj.Proj(init='EPSG:3646', preserve_units=True)

In [11]:
def load_csv(fname):
    falsies = ['No', 'NO','no', ]
    trusies = ['YES', 'Yes', 'yes']
    df = pd.read_csv(fname,
                     true_values=trusies,
                     false_values=falsies,
                     parse_dates=[0],
                     low_memory=False,
                     )
    return df
    

In [12]:
def transform_coordinates(df, projection):
    # 0 values in X and Y columns are invalid - replace with NaN so we can fix erronous projections
    df.X.replace(0, np.nan, inplace=True)
    df.Y.replace(0, np.nan, inplace=True)
    X = np.array(df.X)
    Y = np.array(df.Y)
    
    lons, lats = projection(X, Y, inverse=True)
    # Projections that don't make sense return 1e+30 rather than erroring.
    # We will replace these with NaNs
    lons[lons == 1e+30] = np.nan
    lats[lats == 1e+30] = np.nan
    df['lon'] = lons
    df['lat'] = lats
    # Get rid of old columns
    df.drop(columns=['X', 'Y'], inplace=True)
    
    return df

In [13]:
# def transform_coordinates_to_epsg4326(df, projection, outprojection):
#     # 0 values in X and Y columns are invalid - replace with NaN so we can fix erronous projections
#     df.X.replace(0, np.nan, inplace=True)
#     df.Y.replace(0, np.nan, inplace=True)
#     X = np.array(df.X)
#     Y = np.array(df.Y)
    
#     lons, lats = projection(X, Y, inverse=True)
#     # Projections that don't make sense return 1e+30 rather than erroring.
#     # We will replace these with NaNs
#     lons[lons == 1e+30] = np.nan
#     lats[lats == 1e+30] = np.nan
#     df['lon'] = lons
#     df['lat'] = lats
#     # Transform to EPSG 4326
#     (df['lon'], df['lat']) = transform(projection, outprojection, (df['lon'], df['lat']))
#     # Get rid of old columns
#     df.drop(columns=['X', 'Y'], inplace=True)
    
#     return df

In [14]:
def cleanse_data(df):
    # Get the first number and use that as the new value
    df['num_campers'] = df['How many campers would you estimate are there?'].str.extract(r'(\d+)', expand=False)
    df.drop(columns=['How many campers would you estimate are there?'], inplace=True)
    return df

#This needs to be fixed using the code below. Will do this later.

In [15]:
CSV_FILES = ['../data/onepointcontactreports/Comstock April 2018-November 25_2018.csv']

In [16]:
for fname in CSV_FILES:
    df = load_csv(fname)
    completed = transform_coordinates(df, or_north_plain)
    new_name = '_processed.'.join(fname.split('2018.'))
    df.to_csv(new_name, index=False)

In [17]:
# for fname in CSV_FILES:
#     df = load_csv(fname)
#     completed = transform_coordinates_to_epsg4326(df, or_north_plain, wgs84)
#     new_name = '_processed_epsg4326.'.join(fname.split('2018.'))
#     df.to_csv(new_name, index=False)

# Playground

## *TODO*
- I think extracting column W, in general, is pretty safe, since the question was so direct. With multiple numbers, taking an average of them seems reasonable. We’d need to think about what to do with “+” as many people do that, or say “more than”. My guess is leaving it at the lower bound is fine
-  the CSV you exported seems to have a funny format for column W.
- Some of the numbers are printed as dates. If you look at row 2 (first data row, excluding header) and column W - the one asking “How many campers would you estimate are there?” - then you’ll see it listed as Apr-5 rather than a number. 
- If I go through the fields tonight and select the ones to keep, would you be able to extract the numbers from column W and put them in a new column so I can join that field (via report date) to the other file?
- In terms of keeping columns, I think any qualitative information given about the campsite should stay (i.e. starting with the field “How long has the campsite been there” through the lat/longs
I think we should keep Date and ID
Everything inbetween can likely be reduced to a single column with a street address in it, since it was an either/or sort of thing (sometimes there are multiple fields filled in, but if we go most specific to least specific and exclude NAs, it should work fine)
- "Column W" = “How many campers would you estimate are there?”

#BREAK
- At this point, I have deleted the unwanted columns (manually, in Excel) for the old_processed and new_processed .csvs that were generated above. 
- I'm going to change the name of CSV_FILES to the new filenames that I modified outside of this script and proceed with the next few steps.

In [18]:
CSV_FILES = ['../data/onepointcontactreports/Comstock April 2018-November 25__processed.csv', '../data/onepointcontactreports/campsite_reports_final_HackOR.csv']
print(CSV_FILES)

['../data/onepointcontactreports/Comstock April 2018-November 25__processed.csv', '../data/onepointcontactreports/campsite_reports_final_HackOR.csv']


In [19]:
df = load_csv(CSV_FILES[0])
df2 = load_csv(CSV_FILES[1])

In [20]:
df.head()
print("df2 is {}, df is {}".format(len(df2['Item ID']), len(df['Item ID'])))
# df.to_csv("test_df.csv")

df2 is 32812, df is 21527


In [21]:
df2.drop('Unnamed: 0', axis=1, inplace=True)
df2.head()

,Date Created,Item ID,"Where is the campsite/concern located? Please provide an address if possible. If no address is available, please provide a complete description of the site's location.","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (User Specified)","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (System Verified)",How long has the site been there?,Is the site occupied?,How many campers would you estimate are there?,Are children present?,Are dogs present?,...,Repeated instances of overly aggressive behavior from campers,"Public intoxication and/or conspicuous drug use (If it is an emergency, please call 9-1-1.)",Campsite obstructs public right-of-way,Misuse of public spaces,Structures or tents present,Excessive trash and/or biohazards,Damage to the environment,lon,lat,number_of_campers
0,2018-03-31 20:58:00,1367525.0,Essex Park,3804 se 79th,7811 SE CENTER ST,a while,yes,NaN,False,False,...,False,False,True,True,False,True,False,-122.583146,45.494409,NaN
1,2018-03-31 20:45:00,1367524.0,Max Stop,SE Pershing st & SE 17th ave,NaN,5 Days,yes,1,False,False,...,False,False,True,True,True,True,False,-122.648198,45.499296,1.0
2,2018-03-31 20:14:00,1367522.0,"NE Halsey and 68th, between the freeway offram...",NE Halsey and 68th Pl,NaN,NaN,yes,8,NaN,NaN,...,False,False,False,True,True,True,True,-122.592590,45.533459,8.0
3,2018-03-31 19:38:00,1367515.0,Private Property,NE 74th and NE Roselawn,NaN,2 days,yes,2,False,NaN,...,False,False,False,True,True,False,False,-122.585585,45.560331,2.0
4,2018-03-31 19:23:00,1367514.0,ODOT I-84,NaN,1400-1449 NE 68TH AVE,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,-122.592797,45.533121,NaN


In [22]:
dfs = [df, df2]
# dfc = pd.concat(dfs, ignore_index=True, sort=False)
dfc = pd.concat(dfs, ignore_index=False, sort=False)
# dfc.head(8)
len(dfc['Item ID'])

54339

In [23]:
len1 = len(df['Date Created'])
len2 = len(df2['Date Created'])
len3 =len(dfc['Date Created'])

print(len1+len2)
print(len1, len2, len3)

#Looks like they concat'd successfully
dfc.to_csv("test_dfc.csv")

54339
21527 32812 54339


In [24]:
s = dfc['How many campers would you estimate are there?']
s.head(40)


0                                     5
1                                   NaN
2                                   NaN
3                                   NaN
4                                   NaN
5                                    15
6                                   NaN
7                                   NaN
8                                   NaN
9                                     6
10                                  NaN
11                             Total 13
12                                  NaN
13                                  NaN
14                               15-Oct
15                                    1
16                                   15
17                                  NaN
18                                    1
19                                  NaN
20                                    1
21                                  NaN
22                                  NaN
23    ? Not sure, looks like 5 at least
24                                    1


In [25]:
#Convert any numbers that are written out into integers
islongint = s.str.match('one', na=False)
islongint
#Locate where the 'one' text is
#s.loc[s.str.match('one') == True]
#islongint.loc[731]

dfc.loc[islongint, 'How many campers would you estimate are there?'] = '1'
dfc.loc[731]

,Date Created,Date Received,Item ID,Status,Follow-Ups,"Where is the campsite/concern located? Please provide an address if possible. If no address is available, please provide a complete description of the site's location.","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (User Specified)","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (System Verified)",Address ID,Property ID,...,"Public intoxication and/or conspicuous drug use (If it is an emergency, please call 9-1-1.)",Campsite obstructs public right-of-way,Misuse of public spaces,Structures or tents present,Excessive trash and/or biohazards,Damage to the environment,Is there anything else we should know? When do you see people present?,lon,lat,number_of_campers
731,2018-11-09 09:37:00,NaN,1498723.0,Open,1.0,SWC,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,"Tent, cart, trash on Springwater",-122.557769,45.476033,NaN
731,2018-03-20 09:56:00,NaN,1361285.0,NaN,NaN,Burnside between 3rd and 4th Ave,4 nw 3rd Ave,400-499 W BURNSIDE ST,NaN,NaN,...,True,True,True,True,True,True,NaN,-122.674910,45.523160,NaN


In [26]:
islongint = s.str.match('two', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '2'

islongint = s.str.match('three', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '3'

islongint = s.str.match('four', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '4'

islongint = s.str.match('five', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '5'

islongint = s.str.match('six', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '6'

islongint = s.str.match('seven', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '7'

islongint = s.str.match('eight', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '8'

islongint = s.str.match('nine', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '9'

islongint = s.str.match('r"ten\s"', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '10'

islongint = s.str.match('fifteen', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '15'

islongint = s.str.match('twenty', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '20'


In [27]:
s.loc[s.str.match('one') == True]

26                                                     one
896                                                    one
2598                                               one man
2897                                              one tent
3247                                         one maybe two
3679                                                   one
4128                                                   one
4465                                                   one
5105     one name is Norman and has huge piles on shopp...
5120                               one definitely, maybe 2
6556                                            one or two
10112                                                  one
10327                    one now, but there have been more
10461                                                  one
10565                                                  one
10795                                                  one
11553                                                  o

In [28]:
# Number of rows without any digit in the 'How many campers...' column
s.str.extract(r"(\d+)", expand=False).isnull().sum()

33245

In [29]:
#help(s.str.extract)
s.head(20)
#s.loc[17385]

0            5
1          NaN
2          NaN
3          NaN
4          NaN
5           15
6          NaN
7          NaN
8          NaN
9            6
10         NaN
11    Total 13
12         NaN
13         NaN
14      15-Oct
15           1
16          15
17         NaN
18           1
19         NaN
Name: How many campers would you estimate are there?, dtype: object

In [30]:
ROW_WITH_MULTIPLE_NUMBERS = 17385
s.loc[ROW_WITH_MULTIPLE_NUMBERS]

17385      NaN
17385    15-20
Name: How many campers would you estimate are there?, dtype: object

In [31]:
# First pull out any rows with months in the string, replace with NaN (K)
#(\d+-Apr$)   This will pull out specifically the month of April. 
#(\d{1,2})-([A-Z][a-z]{2}) pulls out all months. 

isbaddate = s.str.match(r"(\d{1,2})-([A-Z][a-z]{2})", na=False)
isbaddate

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14        True
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25        True
26       False
27       False
28       False
29       False
         ...  
32782    False
32783    False
32784    False
32785    False
32786    False
32787    False
32788    False
32789    False
32790    False
32791    False
32792    False
32793    False
32794    False
32795    False
32796    False
32797    False
32798    False
32799    False
32800    False
32801    False
32802    False
32803    False
32804    False
32805    False
32806    False
32807    False
32808    False
32809    False
32810    False
32811    False
Name: How many campers would you estimate are there?, Length: 54339, dtype: bool

In [32]:
#Check out loc, iloc, and ix

In [33]:
dfc.loc[isbaddate, 'How many campers would you estimate are there?'] = np.nan




In [34]:
dfc


,Date Created,Date Received,Item ID,Status,Follow-Ups,"Where is the campsite/concern located? Please provide an address if possible. If no address is available, please provide a complete description of the site's location.","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (User Specified)","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (System Verified)",Address ID,Property ID,...,"Public intoxication and/or conspicuous drug use (If it is an emergency, please call 9-1-1.)",Campsite obstructs public right-of-way,Misuse of public spaces,Structures or tents present,Excessive trash and/or biohazards,Damage to the environment,Is there anything else we should know? When do you see people present?,lon,lat,number_of_campers
0,2018-11-19 14:44:00,11/19/18,1503591.0,Open,0.0,7519 N Leonard st,N leonard st and N Chicago st,7410 N CHICAGO AVE,557114.0,NaN,...,True,False,True,True,True,False,Campers are using public space and the racquet...,-122.755294,45.591748,NaN
1,2018-11-19 14:35:00,11/19/18,1503582.0,Open,0.0,test,to,1220 SW 5TH AVE,646159.0,R246102,...,False,False,False,False,False,False,NaN,-122.679057,45.514999,NaN
2,2018-11-19 14:29:00,NaN,1503576.0,Open,0.0,NaN,NaN,NaN,0.0,NaN,...,False,False,False,False,False,False,Large and growing camp along soundwall on east...,-122.565987,45.556555,NaN
3,2018-11-19 14:26:00,NaN,1503571.0,Open,0.0,NaN,NaN,700-799 NE BROADWAY,0.0,NaN,...,False,False,False,False,False,False,Camp on or next to sidewalk on Broadway at NE ...,-122.658411,45.534961,NaN
4,2018-11-19 14:24:00,NaN,1503567.0,Open,0.0,NaN,NaN,0-0 SW MONTGOMERY ST-I405 FWY RAMP,0.0,NaN,...,False,False,False,False,False,False,Camps and remains of camps in median on west s...,-122.689916,45.512986,NaN
5,2018-11-19 14:23:00,11/19/18,1503563.0,Open,0.0,"I-84 off-ramp at SE 57th, between the ramp and...",SE Hoyt and SE 57th,0-0 NE I84 FWY-GLISAN ST RAMP,0.0,NaN,...,True,True,True,True,True,True,Constantly walking back and forth across the f...,-122.604919,45.527436,NaN
6,2018-11-19 14:22:00,NaN,1503560.0,Open,0.0,NaN,NaN,0-0 NE I84 FWY-GLISAN ST RAMP,0.0,NaN,...,False,False,False,False,False,False,Tent city in median at 58th aveneue exit from ...,-122.604756,45.527441,NaN
7,2018-11-19 14:21:00,NaN,1503556.0,Open,0.0,NaN,NaN,8130-8199 NE HALSEY ST-HALSEY ST RAMP,0.0,NaN,...,False,False,False,False,False,False,Camp under Halsey overpass on west side of NE ...,-122.579237,45.534465,NaN
8,2018-11-19 14:19:00,NaN,1503554.0,Open,0.0,NaN,NaN,16-23 SE MORRISON BRG-WATER AVE RAMP,0.0,NaN,...,False,False,False,False,False,False,Massive tent city has formed under overpass. F...,-122.666672,45.516523,NaN
9,2018-11-19 14:19:00,11/19/18,1503552.0,Open,0.0,SE 83rd Avenue between SE Bush St. and Eastpor...,SE 83rd & Bush,3800-3899 SE 83RD AVE,0.0,NaN,...,True,True,True,True,True,True,NaN,-122.577859,45.494520,NaN


In [35]:
# Pull out any numbers in that string
temp = s.str.extractall(r"(?P<Numbers>\d+)")
temp


,,Numbers
,match,
0,0,5
5,0,15
9,0,6
11,0,13
14,0,15
15,0,1
16,0,15
18,0,1
20,0,1


In [36]:
temp2 = temp.Numbers.astype(float)

#temp2.loc[25].mean()
#THIS is correct.

In [37]:
temp2.index.names = ['record','match']
temp2.index.names

temp3 = temp2.groupby(level=['record']).mean()
temp3

record
0          5.0
1          1.0
2          8.0
3          2.0
5         15.0
9          6.0
10         4.0
11         8.5
14        15.0
15         1.0
16         8.0
17         4.5
18         1.0
20         1.0
21         2.0
23         5.0
24         1.0
25         5.0
27         3.0
28         1.0
29        10.0
31         3.0
33         7.0
34         3.0
35         2.0
37         3.5
38        10.0
40         3.0
41         3.0
43         5.0
         ...  
32736      8.0
32739      7.0
32740     30.0
32741      7.5
32743     10.0
32744     10.0
32745    200.0
32747      6.0
32752      3.0
32753      2.0
32764     10.0
32767      6.0
32768      1.0
32772      3.0
32776      6.0
32777     20.0
32784      2.0
32785      2.0
32791      1.0
32796      1.0
32797      2.0
32801      5.0
32802      2.0
32803     30.0
32804      0.0
32805      2.0
32806      3.0
32807      6.0
32808      7.0
32809      3.0
Name: Numbers, Length: 17496, dtype: float64

In [38]:
dfc2 = dfc.assign(number_of_campers=temp3)

In [43]:
dfc2.columns[7]

'If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (System Verified)'

In [48]:
# dfc3 = dfc2.dropna(subset=['lon','lat', dfc2.columns[5], dfc2.columns[6], dfc2.columns[7]])
#NOTE: This is too strong; find the subset that is all 5 together (AND logic), not all 5 (OR logic)
#NOTE: Did not end up subsetting; will do this in the R notebook.
                           
# len(dfc3['number_of_campers'])
len(dfc2['number_of_campers'])

54339

In [49]:
# dfc2.to_csv('../data/onepointcontactreports/campsite_reports_processed_cols_nums.csv')
dfc2.to_csv('../data/onepointcontactreports/campsite_reports_processed_cols_nums_02.csv')

In [74]:
#np.mean(a, axis=0) 
#something like this will take the mean of each row for the indices and return a single-column df (or) array.
#from there, I can append the column to the dataframe. 
#from, I can filter out rows that have NaN either for the # of campers or for hte lat/long to get the final dataset.

In [73]:
#temp2 = temp.Numbers.astype(float)

#temp2.loc[27].mean()
#THIS is correct.

In [14]:
#result = cleanse_data(df)

In [15]:
#result

,Date Created,Item ID,"Where is the campsite/concern located? Please provide an address if possible. If no address is available, please provide a complete description of the site's location.","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (User Specified)","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (System Verified)",X,Y,Address ID,Property ID,State ID,...,What was the make and model of the vehicle?,"What is the license plate, if available?",Repeated instances of overly aggressive behavior from campers,"Public intoxication and/or conspicuous drug use (If it is an emergency, please call 9-1-1.)",Campsite obstructs public right-of-way,Misuse of public spaces,Structures or tents present,Excessive trash and/or biohazards,Damage to the environment,num_campers
0,2017-03-31 21:26:00,1166167.0,along MUP North of SE Flavel and East of 92ndAve,NaN,NaN,0.000,0.000,NaN,NaN,NaN,...,NaN,NaN,False,False,True,True,True,Yes,Yes,5
1,2017-03-31 21:13:00,1166163.0,NaN,NaN,NaN,7671608.498,663720.642,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
2,2017-03-31 21:12:00,1166162.0,NaN,NaN,9690-9699 SE FLAVEL ST,7672165.664,663601.344,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
3,2017-03-31 20:42:00,1166158.0,NaN,NaN,12500-12899 NE SHAVER ST,7681419.287,693844.334,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
4,2017-03-31 20:40:00,1166157.0,NaN,NaN,12500-12899 NE SHAVER ST,7681264.838,693826.702,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
5,2017-03-31 20:37:00,1166156.0,NaN,NaN,12500-12899 NE SHAVER ST,7681237.336,693823.383,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
6,2017-03-31 20:34:00,1166154.0,NaN,NaN,12500-12899 NE SHAVER ST,7681516.553,693829.850,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
7,2017-03-31 20:16:00,1166152.0,NaN,NaN,NaN,7652801.131,674934.195,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
8,2017-03-31 19:41:00,1166149.0,Between NE 66th and 67th on NE Sacramento,2615 NE 58th,NaN,0.000,0.000,NaN,NaN,NaN,...,OR,NaN,False,False,False,True,True,No,Yes,NaN
9,2017-03-31 19:06:00,1166142.0,NaN,NaN,NaN,7672043.826,671335.119,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
